In [1]:
import pandas as pd
import numpy as np


def my_reducer(a):
    return pd.to_datetime(a[0])

reducer = np.vectorize(my_reducer)

In [2]:
spy = pd.read_csv('/Users/arkilic/Desktop/spy.csv', 
                      header=0, sep=' ')
spy_design = pd.read_csv('/Users/arkilic/Desktop/spy_design.csv', 
                         header=0, sep=' ',  parse_dates=True)
spy_indices = spy.index.values
spy_design_indices = spy_design.index.values


spy_indexes = reducer(spy_indices)
spy['TIME'] = spy_indexes

spy_design_indexes = reducer(spy_design_indices)
spy_design['TIME'] = spy_design_indexes

spy = spy.set_index('TIME')
spy_design = spy_design.set_index('TIME')

In [3]:
spy_data = spy.ix[:, 1:6]
spy_data.head()

,SPY.Open,SPY.High,SPY.Low,SPY.Close,SPY.Volume
TIME,,,,,
2008-01-02 09:31:00,146.535,146.645,146.535,146.645,591203.0
2008-01-02 09:32:00,146.645,146.685,146.635,146.685,385457.0
2008-01-02 09:33:00,146.685,146.775,146.680,146.775,343700.0
2008-01-02 09:34:00,146.725,146.755,146.705,146.725,863418.0
2008-01-02 09:35:00,146.705,146.775,146.685,146.775,457500.0


In [4]:
spy_design_data = spy_design.ix[:, 1:7]
spy_design_data.head()


spy = spy_data
spy_design = spy_design_data

In [5]:
spy_design.head()

,returns,returns.WA5,variance,skew,hurst,rets_var
TIME,,,,,,
2008-01-02 09:31:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2008-01-02 09:32:00,0.707107,0.707107,0.707107,-0.707107,0.000000,0.707107
2008-01-02 09:33:00,1.034910,1.031794,1.027297,-0.901304,0.000000,1.133169
2008-01-02 09:34:00,-1.177622,-0.712844,1.407897,1.277819,0.000000,-1.366479
2008-01-02 09:35:00,0.451147,-0.181499,0.593341,-1.669053,-1.788854,0.854902


In [6]:
def diff_xts(x_ts, lag=1, differences=1, arithmetic=True):
    # Uses something along the lines of numpy.diff for lag
    """
    Parameters
    ----------
    x_ts: pd.DataFrame
        Raw data
    lag: int
        period to difference over
    differences: int
        order of differencing
    arithmetic: bool
        should arithmetic or geometric differencing be used
    """
    if lag != 1:
        raise NotImplementedError('Period other than 1 has not beein implemented yet')
    res = np.zeros(x_ts.shape[0])
    res[1:] = np.diff(x_ts.ix[:, 1])
    return  res# vectorized diff over given period

In [18]:
def run_returns(x_ts, col_umn = 3):
    if len(x_ts.columns) == 5:
        re_turns = 0.5 * (x_ts[:]["Bid.Price"] + x_ts[:]["Ask.Price"])
    else:
        re_turns = x_ts[x_ts.columns[::col_umn]]
    ind_diff = np.ones(x_ts.shape[0])    
    ind_diff[1:] = np.multiply(np.diff(x_ts.index.values), 10e-10)
    x_ts['returns'] = 86400 * diff_xts(re_turns) / ind_diff
    return x_ts

In [21]:
returns_running = run_returns(spy)['returns']

In [34]:
returns_running.head()

TIME
2008-01-02 09:31:00      0.0
2008-01-02 09:32:00     57.6
2008-01-02 09:33:00    129.6
2008-01-02 09:34:00    -72.0
2008-01-02 09:35:00     72.0
Name: returns, dtype: float64

In [10]:
%timeit run_returns(spy)

100 loops, best of 3: 15.3 ms per loop


In [72]:
def lag_xts(x_ts, k=-1, **kwargs):
    n_rows = x_ts.shape[0]
    fir_st = x_ts[0]
    la_st = x_ts[n_rows-1]
    x_ts = x_ts.shift(periods=k, **kwargs)
    if (k > 0):
        print(fir_st)
        x_ts[0:k] = fir_st
    else:
        x_ts[n_rows-1] = la_st
    return x_ts

In [77]:
returns_advanced = lag_xts(returns_running)
returns_advanced.name = 'returns_advanced'
print(returns_advanced)

TIME
2008-01-02 09:31:00     57.6
2008-01-02 09:32:00    129.6
2008-01-02 09:33:00    -72.0
2008-01-02 09:34:00     72.0
2008-01-02 09:35:00    -57.6
2008-01-02 09:36:00   -201.6
2008-01-02 09:37:00    -43.2
2008-01-02 09:38:00    187.2
2008-01-02 09:39:00    115.2
2008-01-02 09:40:00     28.8
2008-01-02 09:41:00     28.8
2008-01-02 09:42:00    165.6
2008-01-02 09:43:00     21.6
2008-01-02 09:44:00      0.0
2008-01-02 09:45:00    -57.6
2008-01-02 09:46:00    -28.8
2008-01-02 09:47:00     28.8
2008-01-02 09:48:00    -72.0
2008-01-02 09:49:00      0.0
2008-01-02 09:50:00   -100.8
2008-01-02 09:51:00      0.0
2008-01-02 09:52:00     14.4
2008-01-02 09:53:00     72.0
2008-01-02 09:54:00     14.4
2008-01-02 09:55:00    -72.0
2008-01-02 09:56:00   -100.8
2008-01-02 09:57:00   -172.8
2008-01-02 09:58:00    115.2
2008-01-02 09:59:00     43.2
2008-01-02 10:00:00   -763.2
                       ...  
2014-05-19 15:32:00     43.2
2014-05-19 15:33:00    -14.4
2014-05-19 15:34:00     72.0
2014-05-1